In [4]:
import pandas as pd
import numpy as np

In [5]:
file_path = 'D:\Book_Market_Insights\data\Books_Data_Clean.csv'
try:
    df = pd.read_csv(file_path)
    print(f"Dataset loaded successfully from: {file_path}")
    print("\nInitial Head:")
    print(df.head())
    print("\nInitial Info:")
    df.info()
except FileNotFoundError:
    print(f"Error: The file '{file_path}' was not found. Please check the path and filename.")
    exit()

Dataset loaded successfully from: D:\Book_Market_Insights\data\Books_Data_Clean.csv

Initial Head:
   index  Publishing Year                        Book Name  \
0      0           1975.0                          Beowulf   
1      1           1987.0                 Batman: Year One   
2      2           2015.0                Go Set a Watchman   
3      3           2008.0  When You Are Engulfed in Flames   
4      4           2011.0         Daughter of Smoke & Bone   

                                              Author language_code  \
0                             Unknown, Seamus Heaney         en-US   
1  Frank Miller, David Mazzucchelli, Richmond Lew...           eng   
2                                         Harper Lee           eng   
3                                      David Sedaris         en-US   
4                                       Laini Taylor           eng   

  Author_Rating  Book_average_rating  Book_ratings_count          genre  \
0        Novice                 

In [6]:
df_cleaned = df.copy()

In [ ]:
print("\n--- Initial Data Inspection ---")
print("\nMissing values before cleaning:")
print(df_cleaned.isnull().sum())
print("\nDescriptive statistics for numerical columns:")
print(df_cleaned.describe())
print("\nUnique values in key categorical columns:")
print(f"Unique language codes: {df_cleaned['language_code'].nunique()} -> {df_cleaned['language_code'].unique()}")
print(f"Unique Author_Ratings: {df_cleaned['Author_Rating'].nunique()} -> {df_cleaned['Author_Rating'].unique()}")
print(f"Unique genres: {df_cleaned['genre'].nunique()} -> {df_cleaned['genre'].unique()}")


--- Initial Data Inspection ---

Missing values before cleaning:
index                   0
Publishing Year         1
Book Name              23
Author                  0
language_code          53
Author_Rating           0
Book_average_rating     0
Book_ratings_count      0
genre                   0
gross sales             0
publisher revenue       0
sale price              0
sales rank              0
Publisher               0
units sold              0
dtype: int64

Descriptive statistics for numerical columns:
             index  Publishing Year  Book_average_rating  Book_ratings_count  \
count  1070.000000      1069.000000          1070.000000         1070.000000   
mean    534.500000      1971.377923             4.007000        94909.913084   
std     309.026698       185.080257             0.247244        31513.242518   
min       0.000000      -560.000000             2.970000        27308.000000   
25%     267.250000      1985.000000             3.850000        70398.000000   
50% 

In [10]:
if df_cleaned['Publishing Year'].isnull().any():
    mode_year = df_cleaned['Publishing Year'].mode()[0]
    print(f"\nFilling missing 'Publishing Year' with mode: {int(mode_year)}")
    df_cleaned['Publishing Year'] = df_cleaned['Publishing Year'].fillna(mode_year)
df_cleaned['Publishing Year'] = df_cleaned['Publishing Year'].astype(int)
print(f"Converted 'Publishing Year' to int. Min: {df_cleaned['Publishing Year'].min()}, Max: {df_cleaned['Publishing Year'].max()}")


Converted 'Publishing Year' to int. Min: -560, Max: 2016


In [9]:
df_cleaned['Primary Author'] = df_cleaned['Author'].apply(lambda x: str(x).split(',')[0].strip())
print(f"\nCreated 'Primary Author' column. Example: '{df_cleaned['Author'].iloc[0]}' -> '{df_cleaned['Primary Author'].iloc[0]}'")



Created 'Primary Author' column. Example: 'Unknown, Seamus Heaney' -> 'Unknown'


In [11]:
initial_rows = df_cleaned.shape[0]
df_cleaned.dropna(subset=['Book Name'], inplace=True)
rows_after_book_name_drop = df_cleaned.shape[0]
print(f"\nDropped {initial_rows - rows_after_book_name_drop} rows due to missing 'Book Name'.")



Dropped 23 rows due to missing 'Book Name'.


In [12]:
df_cleaned['language_code'] = df_cleaned['language_code'].fillna('Unknown')
print(f"Filled missing 'language_code' values with 'Unknown'. Unique language codes now: {df_cleaned['language_code'].unique()}")


Filled missing 'language_code' values with 'Unknown'. Unique language codes now: ['en-US' 'eng' 'Unknown' 'spa' 'en-GB' 'fre' 'en-CA' 'nl' 'ara']


In [13]:
numerical_cols = ['Book_average_rating', 'Book_ratings_count', 'gross sales', 'publisher revenue', 'sale price', 'sales rank', 'units sold']
for col in numerical_cols:
    df_cleaned[col] = pd.to_numeric(df_cleaned[col], errors='coerce')
    if df_cleaned[col].isnull().any():
        print(f"Warning: Missing values found in '{col}' after numeric conversion. Filling with 0 for now.")
        df_cleaned[col] = df_cleaned[col].fillna(0)


In [14]:
df_cleaned['Total Revenue'] = df_cleaned['gross sales']
df_cleaned['Publisher Profit'] = df_cleaned['publisher revenue']


In [15]:
df_cleaned['Rating Category'] = pd.cut(
    df_cleaned['Book_average_rating'],
    bins=[0, 3.0, 3.5, 4.0, 4.5, 5.0],
    labels=['Poor (<3.0)', 'Average (3.0-3.5)', 'Good (3.5-4.0)', 'Very Good (4.0-4.5)', 'Excellent (4.5-5.0)'],
    right=True,
    include_lowest=True 
)
print(f"\nCreated 'Rating Category'. Example: {df_cleaned['Rating Category'].head()}")



Created 'Rating Category'. Example: 0      Average (3.0-3.5)
1    Very Good (4.0-4.5)
2      Average (3.0-3.5)
3    Very Good (4.0-4.5)
4    Very Good (4.0-4.5)
Name: Rating Category, dtype: category
Categories (5, object): ['Poor (<3.0)' < 'Average (3.0-3.5)' < 'Good (3.5-4.0)' < 'Very Good (4.0-4.5)' < 'Excellent (4.5-5.0)']


In [16]:
df_cleaned['genre'] = df_cleaned['genre'].replace('genre fiction', 'Fiction')
print(f"\nStandardized 'genre'. Unique genres now: {df_cleaned['genre'].unique()}")



Standardized 'genre'. Unique genres now: ['Fiction' 'fiction' 'nonfiction' 'children']


In [17]:
print("\n--- After Cleaning and Feature Engineering ---")
print("\nCleaned DataFrame Info:")
df_cleaned.info()
print("\nMissing values after cleaning:")
print(df_cleaned.isnull().sum())
print("\nCleaned Head:")
print(df_cleaned.head())


--- After Cleaning and Feature Engineering ---

Cleaned DataFrame Info:
<class 'pandas.core.frame.DataFrame'>
Index: 1047 entries, 0 to 1069
Data columns (total 19 columns):
 #   Column               Non-Null Count  Dtype   
---  ------               --------------  -----   
 0   index                1047 non-null   int64   
 1   Publishing Year      1047 non-null   int32   
 2   Book Name            1047 non-null   object  
 3   Author               1047 non-null   object  
 4   language_code        1047 non-null   object  
 5   Author_Rating        1047 non-null   object  
 6   Book_average_rating  1047 non-null   float64 
 7   Book_ratings_count   1047 non-null   int64   
 8   genre                1047 non-null   object  
 9   gross sales          1047 non-null   float64 
 10  publisher revenue    1047 non-null   float64 
 11  sale price           1047 non-null   float64 
 12  sales rank           1047 non-null   int64   
 13  Publisher            1047 non-null   object  
 14  unit

In [19]:
output_file_path = '../data/cleaned_books_data_for_powerbi.csv'
df_cleaned.to_csv(output_file_path, index=False)
print(f"\nCleaned data saved to: {output_file_path}")



Cleaned data saved to: ../data/cleaned_books_data_for_powerbi.csv
